### Soham Shah
### J059

In [34]:
import numpy as np
import pandas as pd
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
pd.set_option("display.max_colwidth", 200)
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
%matplotlib inline

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ishanishah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
data = pd.read_csv('papers.csv')

In [26]:
data = df.dropna()

In [27]:
data.head(1)

,id,year,title,event_type,pdf_name,abstract,paper_text
4261,4857,2013,Scalable Influence Estimation in Continuous-Time Diffusion Networks,Oral,4857-scalable-influence-estimation-in-continuous-time-diffusion-networks.pdf,"If a piece of information is released from a media site, can it spread, in 1 month, to a million web pages? This influence estimation problem is very challenging since both the time-sensitive natu...",Scalable Influence Estimation in\nContinuous-Time Diffusion Networks\nNan Du?\nLe Song?\nManuel Gomez-Rodriguez?\nHongyuan Zha?\n?\nGeorgia Institute of Technology\nMPI for Intelligent Systems?\nd...


In [28]:
data.isnull().sum()

id            0
year          0
title         0
event_type    0
pdf_name      0
abstract      0
paper_text    0
dtype: int64

In [29]:
data_text = data[['paper_text']]
data_text['index'] = data_text.index
documents = data_text

In [31]:
documents.head(1)

,paper_text,index
4261,Scalable Influence Estimation in\nContinuous-Time Diffusion Networks\nNan Du?\nLe Song?\nManuel Gomez-Rodriguez?\nHongyuan Zha?\n?\nGeorgia Institute of Technology\nMPI for Intelligent Systems?\nd...,4261


In [32]:
len(documents)

2422

In [35]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [37]:
stemmer = SnowballStemmer('english')

In [39]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

words = []
for word in doc_sample.split(' '):
    words.append(word)

In [41]:
processed_docs = documents['paper_text'].map(preprocess)

In [42]:
processed_docs[:10]

4261    [scalabl, influenc, estim, continu, time, diffus, network, song, manuel, gomez, rodriguez, hongyuan, georgia, institut, technolog, intellig, system, dunan, gatech, lsong, gatech, manuelgr, gatech,...
4262    [adapt, anonym, match, krzysztof, choromanski, columbia, univers, columbia, toni, jebara, columbia, univers, columbia, tang, columbia, univers, columbia, abstract, adapt, anonym, problem, formal, ...
4263    [exact, stabl, recoveri, pairwis, interact, tensor, shouyuan, chen, michael, irwin, king, chines, univers, hong, kong, sychen, king, cuhk, zenglin, purdu, univers, purdu, abstract, tensor, complet...
4265    [matrix, factor, binari, compon, martin, slawski, matthia, hein, pavlo, lutsik, saarland, univers, hein, saarland, lutsik, saarland, abstract, motiv, applic, comput, biolog, consid, rank, matrix, ...
4266    [complex, approxim, binari, evid, lift, infer, broeck, adnan, darwich, scienc, depart, univers, california, angel, guyvdb, darwich, ucla, abstract, lift, infer,

In [44]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [45]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 absolut
1 abstract
2 accumul
3 accur
4 accuraci
5 achiev
6 acknowledg
7 actual
8 acycl
9 adapt
10 add


In [46]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=len(documents))

In [47]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [48]:
bow_doc_4310 = bow_corpus[10]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 2 ("accur") appears 1 time.
Word 6 ("address") appears 1 time.
Word 7 ("adopt") appears 9 time.
Word 14 ("appendix") appears 2 time.
Word 19 ("argument") appears 2 time.
Word 21 ("artifici") appears 3 time.
Word 29 ("binari") appears 3 time.
Word 31 ("cambridg") appears 1 time.
Word 32 ("captur") appears 7 time.
Word 37 ("chen") appears 1 time.
Word 40 ("collect") appears 1 time.
Word 44 ("context") appears 1 time.
Word 59 ("difficult") appears 2 time.
Word 69 ("easi") appears 1 time.
Word 70 ("easili") appears 1 time.
Word 77 ("entir") appears 2 time.
Word 85 ("exponenti") appears 2 time.
Word 95 ("flexibl") appears 1 time.
Word 97 ("formul") appears 7 time.
Word 101 ("furthermor") appears 1 time.
Word 102 ("gain") appears 2 time.
Word 112 ("greater") appears 1 time.
Word 120 ("heurist") appears 3 time.
Word 121 ("hide") appears 3 time.
Word 130 ("infinit") appears 1 time.
Word 136 ("intuit") appears 2 time.
Word 137 ("invari") appears 3 time.
Word 138 ("investig") appears 4 time

### TF-IDF 

In [49]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [50]:
corpus_tfidf = tfidf[bow_corpus]

In [51]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.013495872340591691),
 (1, 0.010897171360212933),
 (2, 0.014759341245638196),
 (3, 0.004329001016622904),
 (4, 0.014002903868499027),
 (5, 0.013707262510466781),
 (6, 0.003491164652545357),
 (7, 0.018921712939585653),
 (8, 0.014747783868130289),
 (9, 0.014199757922470156),
 (10, 0.008890289898277797),
 (11, 0.014235520827159736),
 (12, 0.029185644438191726),
 (13, 0.007272646610429487),
 (14, 0.04205201931799661),
 (15, 0.004324138104783362),
 (16, 0.010968353329539094),
 (17, 0.011825372746208415),
 (18, 0.008227709893299521),
 (19, 0.005837143356061971),
 (20, 0.00559333339324764),
 (21, 0.00736276514069092),
 (22, 0.008648276209566723),
 (23, 0.013766538821677627),
 (24, 0.008204923286253786),
 (25, 0.004876248923399829),
 (26, 0.005153224950881477),
 (27, 0.0571802669671754),
 (28, 0.013327605810536656),
 (29, 0.007312235553026063),
 (30, 0.018585179203472604),
 (31, 0.005423286140575489),
 (32, 0.0034068910201896585),
 (33, 0.006739681958038136),
 (34, 0.02325275923272507),


### LDA Modelling

In [52]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [53]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.020*"regret" + 0.013*"rank" + 0.010*"bandit" + 0.009*"action" + 0.008*"game" + 0.007*"player" + 0.006*"onlin" + 0.006*"round" + 0.006*"arm" + 0.006*"strategi"
Topic: 1 
Words: 0.009*"classifi" + 0.009*"variat" + 0.008*"label" + 0.008*"classif" + 0.007*"posterior" + 0.007*"kernel" + 0.007*"bayesian" + 0.006*"likelihood" + 0.006*"varianc" + 0.006*"convex"
Topic: 2 
Words: 0.009*"submodular" + 0.009*"rank" + 0.004*"deep" + 0.004*"layer" + 0.004*"memori" + 0.004*"rule" + 0.004*"kernel" + 0.004*"graph" + 0.004*"cluster" + 0.004*"convex"
Topic: 3 
Words: 0.010*"word" + 0.008*"domain" + 0.006*"layer" + 0.006*"item" + 0.006*"target" + 0.006*"deep" + 0.005*"classif" + 0.005*"label" + 0.005*"project" + 0.004*"recognit"
Topic: 4 
Words: 0.014*"layer" + 0.009*"neuron" + 0.008*"convolut" + 0.008*"visual" + 0.007*"deep" + 0.006*"activ" + 0.005*"encod" + 0.005*"transform" + 0.005*"attent" + 0.005*"dynam"
Topic: 5 
Words: 0.018*"graph" + 0.014*"cluster" + 0.009*"layer" + 0.009*"kern